In [1]:
import os
import math

import time
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [2]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [3]:
def featureFactory(dataFrame):
    print('Feature Augmentation')
    minute = dataFrame.time%60
    dataFrame['hour'] = dataFrame['time'].div(60).map(int)
#     dataFrame.drop(['time'], axis=1, inplace=True)
    dataFrame['weekday'] = dataFrame['hour'].div(24).map(int)
    dataFrame['month'] = dataFrame['weekday'].div(30).map(int)
#     dataFrame['day']=dataFrame['hour'].div(24).map(int)
    
    dataFrame['year'] = (dataFrame['weekday'].div(365).map(int)+1)*10.0
    dataFrame['hour'] = ((dataFrame.loc[:,'hour'].values%(24)+1)+minute.div(60.0))*(4.0)


    dataFrame['weekday'] = (dataFrame['weekday']%7+1)*3.1
    dataFrame['month'] = (dataFrame['month']%12+1)*2.1
#     dataFrame['dayOfMonth']=(dataFrame['day']%30+1)
    dataFrame['accuracy'] = np.log10(dataFrame['accuracy'])*10.0
    dataFrame['originalIndex'] = np.arange(len(dataFrame))

    print ('processing is done')
f=['x','y', 'accuracy','hour', 'weekday', 'month', 'year']

In [4]:
def amplifier1(dataFrame):
    add_data = dataFrame[dataFrame.hour<10]
    add_data.hour = add_data.hour+96
    return add_data
    
def amplifier2(dataFrame):
    add_data = dataFrame[dataFrame.hour>90]
    add_data.hour = add_data.hour-96
    return add_data

In [5]:
featureFactory(train)
featureFactory(test)
pd.options.mode.chained_assignment = None

ap1=amplifier1(train)
ap2=amplifier2(train)
train=train.append(ap1).append(ap2)

# train=train.append(amplifier1(train))
# train=train.append(amplifier2(train))




Feature Augmentation
processing is done
Feature Augmentation
processing is done


In [6]:
len(ap1)
len(ap2)
train.head()
ap1.head(10)

1818434

3010042

,row_id,x,y,accuracy,time,place_id,hour,weekday,month,year,originalIndex
0,0,0.7941,9.0809,17.323938,470702,8523065625,88.133333,15.5,23.1,10.0,0
1,1,5.9567,4.7968,11.139434,186555,1757726713,57.000000,12.4,10.5,10.0,1
2,2,8.3078,7.0407,18.692317,322648,1137537235,9.866667,3.1,16.8,10.0,2
3,3,7.3665,2.5165,18.129134,704587,6567393236,32.466667,21.7,10.5,20.0,3
4,4,4.0961,1.1307,14.913617,472130,7440663949,87.333333,18.6,23.1,10.0,4


,row_id,x,y,accuracy,time,place_id,hour,weekday,month,year,originalIndex
2,2,8.3078,7.0407,18.692317,322648,1137537235,105.866667,3.1,16.8,10.0,2
58,58,4.5698,4.1206,20.043214,689815,5158095731,103.666667,12.4,8.4,20.0,58
62,62,6.2019,5.4881,24.814426,574643,8673603270,105.533333,3.1,4.2,20.0,62
74,74,1.8616,2.8019,21.238516,769002,4530587605,102.800000,9.3,12.6,20.0,74
139,139,7.6932,2.3443,12.787536,118161,5250105119,105.400000,18.6,6.3,10.0,139
144,144,4.7891,1.2985,16.812412,126743,4490687859,101.533333,15.5,6.3,10.0,144
158,158,4.4052,2.7474,18.129134,483868,7124468757,101.866667,3.1,25.2,10.0,158
170,170,0.3764,6.3198,20.644580,567415,9646456529,103.666667,9.3,4.2,20.0,170
201,201,5.4298,2.9131,18.260748,73465,4108047704,101.666667,9.3,4.2,10.0,201
231,231,3.7208,8.8431,15.797836,668201,8907221664,102.733333,9.3,8.4,20.0,231


In [51]:
# test=pd.read_csv('../input/test.csv')
# featureFactory(test)

In [52]:
# g=train.groupby('place_id')
# places=pd.DataFrame()
# places['xMean']=g['x'].mean()
# places['xStd']=g['x'].std()

# places['yMean']=g['y'].mean()
# places['yStd']=g['y'].std()

# sTime=time.time()

# train=train[(abs(train['y']-places.loc[train['place_id'], 'yMean'].values)<5*places.loc[train['place_id'], 'yStd'].values)\
#          & (abs(train['x']-places.loc[train['place_id'], 'xMean'].values)<5*places.loc[train['place_id'], 'xStd'].values)\
           
#            ]

# print time.time()-sTime

In [7]:
nXRegions=20
nYRegions=40
xMin=0
xMax=10.1
yMin=0
yMax=10.1
nXRegions=nXRegions*int(xMax-xMin)/10
nYRegions=nYRegions*int(yMax-yMin)/10
# train=train[(train['x']<xMax) & (train['x']>=xMin) & (train['y']<yMax) & (train['y']>=yMin)]
# valid=valid[(valid['x']<xMax) & (valid['x']>=xMin) & (valid['y']<yMax) & (valid['y']>=yMin)]

print xMin, xMax, yMin, yMax, nXRegions, nYRegions

0 10.1 0 10.1 20 40


In [8]:
def train_test(modelSerialization=False, resultSerialization=False, model='rf', predictSet=test, th=0, xMargin=0.05, yMargin=0.025):
    
    
    if modelSerialization or resultSerialization:
        base='./'
        folderName=model+'-'+time.strftime('%c')
        folderPath=base+folderName+'/'
        if not os.path.exists(folderPath):
            os.makedirs(folderPath)
            
    f=['x','y', 'accuracy','hour', 'weekday','month', 'year']

        
    count=0
    accuracies=[]
    FBscores=[]
    
    startTime=time.time()
    trainingTimes=[]
    predictionTimes=[]
    
    testNum=0

    roundNum=5

    for xNum, yNum in ((xn, yn) for xn in xrange(nXRegions) for yn in xrange(nYRegions)):

        xStep=round(1.0*(xMax-xMin)/nXRegions,roundNum)
        xStart=round(xMin+xNum*xStep,roundNum)
        xEnd=round(xStart+xStep,roundNum)

        yStep=round(1.0*(yMax-yMin)/nYRegions,roundNum)
        yStart=round(yMin+yNum*yStep,roundNum)
        yEnd=round(yStart+yStep,roundNum)
        

        regionalTrain=train[(train['x']<xEnd+xMargin) & (train['x']>=xStart-xMargin) & (train['y']<yEnd+yMargin) & (train['y']>=yStart-yMargin)]
#         print xStart, xEnd, yStart, yEnd, len(regionalTrain)



        placesCounts=regionalTrain['place_id'].value_counts()
        regionalTrain=regionalTrain.loc[(placesCounts[regionalTrain['place_id'].values]>=th).values]
#        this is no good, neight :tail or -tail:
#         tail=len(regionalTrain)/20
#         latest=regionalTrain.sort_values(by=['time'])[-tail:]
#         regionalTrain=regionalTrain.append(latest)
        
        
#         addition=regionalTrain[regionalTrain['hourInDay']<=4]
#         addition['hourInDay']=addition['hourInDay']+24
#         regionalTrain=regionalTrain.append(addition)
        
#         addition=regionalTrain[regionalTrain['hourInDay']>=20]
#         addition['hourInDay']=addition['hourInDay']-24
#         regionalTrain=regionalTrain.append(addition)


        
        if len(regionalTrain):
            regionalPredictSet=predictSet[(predictSet['x']<xEnd) & (predictSet['x']>=xStart) & (predictSet['y']<yEnd) & (predictSet['y']>=yStart)]
            testNum+=len(regionalPredictSet)

            if model=='knn':
                regionalTrain.loc[:,'x'] *= 490.0
                regionalTrain.loc[:,'y'] *= 980.0
                regionalPredictSet.loc[:,'x'] *= 490.0
                regionalPredictSet.loc[:,'y'] *= 980.0

                
            else:
                regionalTrain.loc[:,'x'] *= 490.0
                regionalTrain.loc[:,'y'] *= 980.0
                regionalPredictSet.loc[:,'x'] *= 490.0
                regionalPredictSet.loc[:,'y'] *= 980.0
                
#                 fMean=regionalTrain[f].mean()
#                 fStd=regionalTrain[f].std()

#                 for ft in f:       
#                     regionalTrain[ft]=(regionalTrain[ft].values-fMean[ft])/(fStd[ft])
#                     regionalPredictSet[ft]=(regionalPredictSet[ft].values-fMean[ft])/(fStd[ft])


      
            le=LabelEncoder()
            regionalTrain['le']=le.fit_transform(regionalTrain['place_id'].values)

#             s_w=1**(15-regionalTrain['day'].values/30.0)
            
            trainingStartTime=time.time()
            if model=='rf':
#                 s_w_rf=3**(-regionalTrain['month'].values*30)
#                 s_w_rf=1.65-regionalTrain['day'].values
                s_w_rf=np.ones(len(regionalTrain))

    #             clf=RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features=None).fit(regionalTrain[f], regionalTrain['le'])   
            
                clf=RandomForestClassifier(n_jobs=-1, n_estimators=150, random_state=0).fit(regionalTrain[f], regionalTrain['le'], sample_weight=s_w_rf)  

            if model=='xgb':
                clf=XGBClassifier(learning_rate=0.02, n_estimators=220, objective='multi:softprob', max_depth=3, seed=0).fit(regionalTrain[f], regionalTrain['le'])
            
            
            if model=='knn':
                def calculate_distance(distances):
                    return distances ** -2
#                 clf=KNeighborsClassifier(n_neighbors=25, weights='distance',metric='manhattan', n_jobs=-1).fit(regionalTrain[f], regionalTrain['le'])
#                 numNeighbors=np.floor(np.sqrt(len(regionalPredictSet))/5.1282).astype(int)
                numNeighbors=np.floor(0.32*np.sqrt(len(regionalPredictSet)))
#                 numNeighbors=36


                clf=KNeighborsClassifier(n_neighbors=numNeighbors, weights=calculate_distance, metric='manhattan', n_jobs=-1).fit(regionalTrain[f], regionalTrain['le'])

         
            trainingTimes.append(time.time()-trainingStartTime)
            
            
            if modelSerialization:
                modelFileName='{:04d}-{}-{}-{}-{}.clf'.format(count, xStart, xEnd, yStart, yEnd)
                modelFilePath=folderPath+modelFileName
                with open(modelFilePath, 'ab+') as fo:
                    pickle.dump(clf, fo, pickle.HIGHEST_PROTOCOL)
            
            
            predictionStartTime=time.time()
            prob=clf.predict_proba(regionalPredictSet[f])
            predictionTimes.append(time.time()-predictionStartTime)
            
            pred=[sorted(zip(p, clf.classes_))[-3:][::-1] for p in prob]
            prediction=le.inverse_transform([zip(*p)[1] for p in pred])
            confidence=[zip(*p)[0] for p in pred]
            
            
                
            fbscoreForSerialization=-1
            if 'place_id' in predictSet.columns:
                regionalAccuracy=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                regionalConfidence=[sum(z)/len(regionalPredictSet) for z in zip(*confidence)]
                regionalConfidence3=sum(regionalConfidence)

                fbAccuracy=0
                fbAccuracy+=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][:1] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/2*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][1:2] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/3*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][2:3] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                FBscores.append(fbAccuracy)
                print 'region {}: {},{} accuracy: {},  fbAccu: {}, confidence: {}:'.format(count, xNum, yNum, regionalAccuracy, fbAccuracy, regionalConfidence3)
                accuracies.append(regionalAccuracy)
                if resultSerialization:
                    fbscoreForSerialization=fbAccuracy
                
            if resultSerialization:
                resultFileName='{:04d}-{}-{}-{}-{}.rst'.format(count, xStart, xEnd, yStart, yEnd)
                resultFilePath=folderPath+resultFileName
                predColumns=zip(*prediction)
                confColumns=zip(*confidence)

                
                if len(predColumns[2])!=len(prediction):
                    print 'missing values',count, len(predColumns[0])
                
                results=pd.DataFrame({'originalIndex': regionalPredictSet['originalIndex'].tolist(),\
                                      'x':regionalPredictSet['x'].tolist(), 'y':regionalPredictSet['y'].tolist(), \
                                      'accuracy':regionalPredictSet['accuracy'].tolist(), 'pred0':predColumns[0], \
                                      'pred1':predColumns[1], 'pred2':predColumns[2], 'conf0': confColumns[0], \
                                      'conf1': confColumns[1], 'conf2': confColumns[2], 'regionalFBScore': [fbscoreForSerialization]*len(regionalPredictSet)})

                results.to_csv(resultFilePath)



        count+=1
        if count%10==0:

            print '{} : total time {} s.'.format(count, time.time()-startTime)
            print 'Average training Time: ', sum(trainingTimes)/len(trainingTimes)
            print 'Average prediction Time: ', sum(predictionTimes)/len(predictionTimes)
            print 'Average FB Score', np.mean(FBscores)
            print 
            
            startTime=time.time()
            trainingTimes=[]
            predictionTimes=[]


    print 
    print np.mean(accuracies)
    print np.var(accuracies)
    print
    print np.mean(FBscores)
    print testNum
    print 'done'
    
    
    
    

In [9]:
train_test(modelSerialization=False, model='knn', resultSerialization=True, predictSet=test, th=5, xMargin=0.05, yMargin=0.025)

/home/zhenjie/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/parallel.py:72: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return [func(*args, **kwargs) for func, args, kwargs in self.items]


10 : total time 17.0417580605 s.
Average training Time:  0.0256668806076
Average prediction Time:  0.404391598701
Average FB Score nan



/home/zhenjie/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


20 : total time 15.6031479836 s.
Average training Time:  0.025309920311
Average prediction Time:  0.416748690605
Average FB Score nan

30 : total time 14.4254410267 s.
Average training Time:  0.0232563257217
Average prediction Time:  0.400178599358
Average FB Score nan

40 : total time 15.7847819328 s.
Average training Time:  0.0233147859573
Average prediction Time:  0.490953016281
Average FB Score nan

50 : total time 16.7254450321 s.
Average training Time:  0.0281741142273
Average prediction Time:  0.431282877922
Average FB Score nan

60 : total time 16.4500801563 s.
Average training Time:  0.0280405521393
Average prediction Time:  0.396517729759
Average FB Score nan

70 : total time 16.2447290421 s.
Average training Time:  0.0269246101379
Average prediction Time:  0.438189268112
Average FB Score nan

80 : total time 15.9879179001 s.
Average training Time:  0.0265630245209
Average prediction Time:  0.422386932373
Average FB Score nan

90 : total time 17.2042870522 s.
Average training

/home/zhenjie/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [10]:
def makeSubmission(resultFolder):
    results=[]
    for f in os.listdir(resultFolder):
        if f.endswith('.rst'):
            fi=resultFolder+'/'+f
            results.append(pd.read_csv(fi))
    results=pd.concat(results)  
        
    results.sort(['originalIndex'], inplace=True)
    return results

In [11]:
results=makeSubmission('../submissions/knn-3.2ratioNeighors/')
predictions=results[['pred0','pred1','pred2']]
submit=pd.DataFrame()
# submit.loc[:,'row_id']=np.arange(len(predictions))
submit.loc[:,'place_id']=predictions[['pred0', 'pred1', 'pred2']].apply(lambda x: ' '.join([str(nx) for nx in x]), axis=1)
submit.loc[:,'row_id']=np.arange(len(predictions))


/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
submit=submit[['row_id','place_id' ]]
submit.set_index('row_id', inplace=True)
submit.to_csv('../submissions/knn-3.2ratioNeighors.csv')
